# Zadanie 1
Pobierz notatniki Joins.dbc, SMJ.dbc i Generator.dbc

Cel zadania: Wykonanie różnych typów jonów i sprawdzenie jaki to ma wpływ na wyniki zapytania. 
Wykonaj połączenia w notatniku Joins.dbc

Dla tych jonów porównaj inner i left and right outer, Left Semi Join, Left Anti Join join 
za każdym razem sprawdź .explain i zobacz jak Spark wykonuje połączenia


In [0]:
from pyspark.sql import SparkSession

# Start Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

# DataFrames creation in PySpark
person = spark.createDataFrame([
    (0, "Bill Chambers", 0, [100]),
    (1, "Matei Zaharia", 1, [500, 250, 100]),
    (2, "Michael Armbrust", 1, [250, 100])
], ["id", "name", "graduate_program", "spark_status"])

graduateProgram = spark.createDataFrame([
    (0, "Masters", "School of Information", "UC Berkeley"),
    (2, "Masters", "EECS", "UC Berkeley"),
    (1, "Ph.D.", "EECS", "UC Berkeley")
], ["id", "degree", "department", "school"])

sparkStatus = spark.createDataFrame([
    (500, "Vice President"),
    (250, "PMC Member"),
    (100, "Contributor")
], ["id", "status"])


In [0]:
person.createOrReplaceTempView("person")
graduateProgram.createOrReplaceTempView("graduateProgram")
sparkStatus.createOrReplaceTempView("sparkStatus")

In [0]:
person = spark.sql("SELECT * FROM person")
person.show()


+---+----------------+----------------+---------------+
| id|            name|graduate_program|   spark_status|
+---+----------------+----------------+---------------+
|  0|   Bill Chambers|               0|          [100]|
|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  2|Michael Armbrust|               1|     [250, 100]|
+---+----------------+----------------+---------------+



In [0]:
graduateProgram = spark.sql("SELECT * FROM graduateProgram")
display(graduateProgram)
graduateProgram.show()

id,degree,department,school
0,Masters,School of Information,UC Berkeley
2,Masters,EECS,UC Berkeley
1,Ph.D.,EECS,UC Berkeley


+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  2|Masters|                EECS|UC Berkeley|
|  1|  Ph.D.|                EECS|UC Berkeley|
+---+-------+--------------------+-----------+



In [0]:
display(spark.sql("SELECT * FROM sparkStatus"))


id,status
500,Vice President
250,PMC Member
100,Contributor


In [0]:
joinExpression = person["graduate_program"] == graduateProgram["id"]


In [0]:
wrongJoinExpression = person["name"] == graduateProgram["school"]


In [0]:
person.join(graduateProgram, joinExpression).show()


+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+



In [0]:
person.join(graduateProgram, wrongJoinExpression).show()


+---+----+----------------+------------+---+------+----------+------+
| id|name|graduate_program|spark_status| id|degree|department|school|
+---+----+----------------+------------+---+------+----------+------+
+---+----+----------------+------------+---+------+----------+------+



In [0]:
joinType = "inner"
join1 = person.join(graduateProgram, joinExpression, joinType)
join1.explain()
join1.show()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [graduate_program#2464L], [id#2470L], Inner
   :- Sort [graduate_program#2464L ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(graduate_program#2464L, 200), ENSURE_REQUIREMENTS, [plan_id=14388]
   :     +- Filter isnotnull(graduate_program#2464L)
   :        +- Scan ExistingRDD[id#2462L,name#2463,graduate_program#2464L,spark_status#2465]
   +- Sort [id#2470L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(id#2470L, 200), ENSURE_REQUIREMENTS, [plan_id=14389]
         +- Filter isnotnull(id#2470L)
            +- Scan ExistingRDD[id#2470L,degree#2471,department#2472,school#2473]


+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bil

In [0]:
joinType = "outer"
join2 = person.join(graduateProgram, joinExpression, joinType)
join2.explain()
join2.show()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [graduate_program#2464L], [id#2470L], FullOuter
   :- Sort [graduate_program#2464L ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(graduate_program#2464L, 200), ENSURE_REQUIREMENTS, [plan_id=14602]
   :     +- Scan ExistingRDD[id#2462L,name#2463,graduate_program#2464L,spark_status#2465]
   +- Sort [id#2470L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(id#2470L, 200), ENSURE_REQUIREMENTS, [plan_id=14603]
         +- Scan ExistingRDD[id#2470L,degree#2471,department#2472,school#2473]


+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|


In [0]:
joinType = "left_outer"
join3 = graduateProgram.join(person, joinExpression, joinType)
join3.explain()
join3.show()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [id#2470L], [graduate_program#2464L], LeftOuter
   :- Sort [id#2470L ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(id#2470L, 200), ENSURE_REQUIREMENTS, [plan_id=14794]
   :     +- Scan ExistingRDD[id#2470L,degree#2471,department#2472,school#2473]
   +- Sort [graduate_program#2464L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(graduate_program#2464L, 200), ENSURE_REQUIREMENTS, [plan_id=14795]
         +- Filter isnotnull(graduate_program#2464L)
            +- Scan ExistingRDD[id#2462L,name#2463,graduate_program#2464L,spark_status#2465]


+---+-------+--------------------+-----------+----+----------------+----------------+---------------+
| id| degree|          department|     school|  id|            name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+----+----------------+----------------+---------------+
|  0|Masters|School of Informa...|UC Berkeley|

In [0]:
joinType = "right"
join4 = person.join(graduateProgram, joinExpression, joinType)
join4.explain()
join4.show()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [graduate_program#2464L], [id#2470L], RightOuter
   :- Sort [graduate_program#2464L ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(graduate_program#2464L, 200), ENSURE_REQUIREMENTS, [plan_id=15022]
   :     +- Filter isnotnull(graduate_program#2464L)
   :        +- Scan ExistingRDD[id#2462L,name#2463,graduate_program#2464L,spark_status#2465]
   +- Sort [id#2470L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(id#2470L, 200), ENSURE_REQUIREMENTS, [plan_id=15023]
         +- Scan ExistingRDD[id#2470L,degree#2471,department#2472,school#2473]


+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|               0|     

In [0]:
joinType = "left_semi"
join5 = graduateProgram.join(person, joinExpression, joinType)
join5.explain()
join5.show()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [id#2470L], [graduate_program#2464L], LeftSemi
   :- Sort [id#2470L ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(id#2470L, 200), ENSURE_REQUIREMENTS, [plan_id=15261]
   :     +- Filter isnotnull(id#2470L)
   :        +- Scan ExistingRDD[id#2470L,degree#2471,department#2472,school#2473]
   +- Sort [graduate_program#2464L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(graduate_program#2464L, 200), ENSURE_REQUIREMENTS, [plan_id=15262]
         +- Project [graduate_program#2464L]
            +- Filter isnotnull(graduate_program#2464L)
               +- Scan ExistingRDD[id#2462L,name#2463,graduate_program#2464L,spark_status#2465]


+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  1|  Ph.D.|                EECS|UC Berkeley|
+---+-------+------------

In [0]:
gradProgram2 = graduateProgram.union(spark.createDataFrame([
    (0, "Masters", "Duplicated Row", "Duplicated School")
], ["id", "degree", "department", "school"]))

gradProgram2.createOrReplaceTempView("gradProgram2")

In [0]:
join6 = gradProgram2.join(person, joinExpression, joinType)
join6.explain()
join6.show()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Union
   :- SortMergeJoin [id#2470L], [graduate_program#2464L], LeftSemi
   :  :- Sort [id#2470L ASC NULLS FIRST], false, 0
   :  :  +- Exchange hashpartitioning(id#2470L, 200), ENSURE_REQUIREMENTS, [plan_id=15525]
   :  :     +- Filter isnotnull(id#2470L)
   :  :        +- Scan ExistingRDD[id#2470L,degree#2471,department#2472,school#2473]
   :  +- Sort [graduate_program#2464L ASC NULLS FIRST], false, 0
   :     +- Exchange hashpartitioning(graduate_program#2464L, 200), ENSURE_REQUIREMENTS, [plan_id=15526]
   :        +- Project [graduate_program#2464L]
   :           +- Filter isnotnull(graduate_program#2464L)
   :              +- Scan ExistingRDD[id#2462L,name#2463,graduate_program#2464L,spark_status#2465]
   +- SortMergeJoin [id#2954L], [graduate_program#2464L], LeftSemi
      :- Sort [id#2954L ASC NULLS FIRST], false, 0
      :  +- Exchange hashpartitioning(id#2954L, 200), ENSURE_REQUIREMENTS, [plan_id=15531]
      :     +-

In [0]:
joinType = "left_anti"
join7 = graduateProgram.join(person, joinExpression, joinType)
join7.explain()
join7.show()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [id#2470L], [graduate_program#2464L], LeftAnti
   :- Sort [id#2470L ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(id#2470L, 200), ENSURE_REQUIREMENTS, [plan_id=16075]
   :     +- Scan ExistingRDD[id#2470L,degree#2471,department#2472,school#2473]
   +- Sort [graduate_program#2464L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(graduate_program#2464L, 200), ENSURE_REQUIREMENTS, [plan_id=16076]
         +- Project [graduate_program#2464L]
            +- Filter isnotnull(graduate_program#2464L)
               +- Scan ExistingRDD[id#2462L,name#2463,graduate_program#2464L,spark_status#2465]


+---+-------+----------+-----------+
| id| degree|department|     school|
+---+-------+----------+-----------+
|  2|Masters|      EECS|UC Berkeley|
+---+-------+----------+-----------+



In [0]:
joinType = "cross"
join8 = graduateProgram.join(person, joinExpression, joinType)
join8.explain()
join8.show()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [id#2470L], [graduate_program#2464L], Cross
   :- Sort [id#2470L ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(id#2470L, 200), ENSURE_REQUIREMENTS, [plan_id=16301]
   :     +- Filter isnotnull(id#2470L)
   :        +- Scan ExistingRDD[id#2470L,degree#2471,department#2472,school#2473]
   +- Sort [graduate_program#2464L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(graduate_program#2464L, 200), ENSURE_REQUIREMENTS, [plan_id=16302]
         +- Filter isnotnull(graduate_program#2464L)
            +- Scan ExistingRDD[id#2462L,name#2463,graduate_program#2464L,spark_status#2465]


+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
| id| degree|          department|     school| id|            name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
|  0|Master

In [0]:
join9 = graduateProgram.crossJoin(person)
join9.explain()
join9.show()

== Physical Plan ==
CartesianProduct
:- *(1) Scan ExistingRDD[id#2470L,degree#2471,department#2472,school#2473]
+- *(2) Scan ExistingRDD[id#2462L,name#2463,graduate_program#2464L,spark_status#2465]


+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
| id| degree|          department|     school| id|            name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
|  0|Masters|School of Informa...|UC Berkeley|  0|   Bill Chambers|               0|          [100]|
|  0|Masters|School of Informa...|UC Berkeley|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  0|Masters|School of Informa...|UC Berkeley|  2|Michael Armbrust|               1|     [250, 100]|
|  2|Masters|                EECS|UC Berkeley|  0|   Bill Chambers|               0|          [100]|
|  2|Masters|                EECS|UC Berkeley|  1|   Matei Zaharia|               1|[500, 250

In [0]:
join10 = graduateProgram.join(person, joinExpression, joinType)
join10.explain()
join10.show()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [id#2470L], [graduate_program#2464L], Cross
   :- Sort [id#2470L ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(id#2470L, 200), ENSURE_REQUIREMENTS, [plan_id=16600]
   :     +- Filter isnotnull(id#2470L)
   :        +- Scan ExistingRDD[id#2470L,degree#2471,department#2472,school#2473]
   +- Sort [graduate_program#2464L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(graduate_program#2464L, 200), ENSURE_REQUIREMENTS, [plan_id=16601]
         +- Filter isnotnull(graduate_program#2464L)
            +- Scan ExistingRDD[id#2462L,name#2463,graduate_program#2464L,spark_status#2465]


+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
| id| degree|          department|     school| id|            name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
|  0|Master

In [0]:
join11 = graduateProgram.crossJoin(person)
join11.explain()
join11.show()

== Physical Plan ==
CartesianProduct
:- *(1) Scan ExistingRDD[id#2470L,degree#2471,department#2472,school#2473]
+- *(2) Scan ExistingRDD[id#2462L,name#2463,graduate_program#2464L,spark_status#2465]


+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
| id| degree|          department|     school| id|            name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
|  0|Masters|School of Informa...|UC Berkeley|  0|   Bill Chambers|               0|          [100]|
|  0|Masters|School of Informa...|UC Berkeley|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  0|Masters|School of Informa...|UC Berkeley|  2|Michael Armbrust|               1|     [250, 100]|
|  2|Masters|                EECS|UC Berkeley|  0|   Bill Chambers|               0|          [100]|
|  2|Masters|                EECS|UC Berkeley|  1|   Matei Zaharia|               1|[500, 250

In [0]:
from pyspark.sql.functions import expr

person.withColumnRenamed("id", "personId") \
    .join(sparkStatus, expr("array_contains(spark_status, id)")).show()

+--------+----------------+----------------+---------------+---+--------------+
|personId|            name|graduate_program|   spark_status| id|        status|
+--------+----------------+----------------+---------------+---+--------------+
|       0|   Bill Chambers|               0|          [100]|100|   Contributor|
|       1|   Matei Zaharia|               1|[500, 250, 100]|500|Vice President|
|       1|   Matei Zaharia|               1|[500, 250, 100]|250|    PMC Member|
|       1|   Matei Zaharia|               1|[500, 250, 100]|100|   Contributor|
|       2|Michael Armbrust|               1|     [250, 100]|250|    PMC Member|
|       2|Michael Armbrust|               1|     [250, 100]|100|   Contributor|
+--------+----------------+----------------+---------------+---+--------------+



Duplikaty

In [0]:
gradProgramDupe = graduateProgram.withColumnRenamed("id", "graduate_program")
gradProgramDupe.show()

+----------------+-------+--------------------+-----------+
|graduate_program| degree|          department|     school|
+----------------+-------+--------------------+-----------+
|               0|Masters|School of Informa...|UC Berkeley|
|               2|Masters|                EECS|UC Berkeley|
|               1|  Ph.D.|                EECS|UC Berkeley|
+----------------+-------+--------------------+-----------+



In [0]:
joinExpr = gradProgramDupe["graduate_program"] == person["graduate_program"]


In [0]:
person.join(gradProgramDupe, joinExpr).show()


+---+----------------+----------------+---------------+----------------+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status|graduate_program| degree|          department|     school|
+---+----------------+----------------+---------------+----------------+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|               0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|               1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|               1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+----------------+-------+--------------------+-----------+



Opcja 1

In [0]:
person.join(gradProgramDupe, joinExpr).select("person.graduate_program").show()


+----------------+
|graduate_program|
+----------------+
|               0|
|               1|
|               1|
+----------------+



Opcja 2 Drop after join

In [0]:
person.join(gradProgramDupe, "graduate_program").select("graduate_program").show()


+----------------+
|graduate_program|
+----------------+
|               0|
|               1|
|               1|
+----------------+



In [0]:
person.join(gradProgramDupe, joinExpr) \
    .drop(person["graduate_program"]) \
    .select("graduate_program").show()

+----------------+
|graduate_program|
+----------------+
|               0|
|               1|
|               1|
+----------------+



In [0]:
joinExpr = person["graduate_program"] == graduateProgram["id"]
person.join(graduateProgram, joinExpr) \
    .drop(graduateProgram["id"]).show()

+---+----------------+----------------+---------------+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| degree|          department|     school|
+---+----------------+----------------+---------------+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+-------+--------------------+-----------+



Opcja 3

In [0]:
gradProgram3 = graduateProgram.withColumnRenamed("id", "grad_id")
joinExpr = person["graduate_program"] == gradProgram3["grad_id"]
person.join(gradProgram3, joinExpr).show()

+---+----------------+----------------+---------------+-------+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status|grad_id| degree|          department|     school|
+---+----------------+----------------+---------------+-------+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|      0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|      1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|      1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+-------+-------+--------------------+-----------+



# Zadanie 2

Cel zadania: usuwanie zduplikowanych kolumn

Użyj Generatora i stwórz dwie duże tabele po 1 milion wierszy i wykonaj dwa typy joinów, inner oraz left. 

Połącz tabele po tych samych kolumnach i użyj jednej metody z wykładów na usunięcie duplikatów. 


In [0]:
import random
from pyspark.sql.functions import col

data1 = [(i, random.randint(100, 999)) for i in range(1, 1000001)]
data2 = [(i, random.randint(100, 999)) for i in range(1, 1000001)]

df1 = spark.createDataFrame(data1, ["id1", "random_value1"])
df2 = spark.createDataFrame(data2, ["id2", "random_value2"])

df1_alias = df1.alias("df1")
df2_alias = df2.alias("df2")



inner_join_df = df1_alias.join(df2_alias, col("df1.id1") == col("df2.id2"), how="inner")

print("Tabela po inner join:")
inner_join_df.show(5)

print("Tabela po inner join po usunięciu duplikatów:")
join_inner_duplicate = inner_join_df.drop("id2")
join_inner_duplicate.show(5)



left_join_df = df1_alias.join(df2_alias, col("df1.id1") == col("df2.id2"), how="left")

print("Tabela po left join:")
left_join_df.show(5)

print("Tabela po left join po usunięciu duplikatów:")
join_left_duplicate = left_join_df.drop("id2")
join_left_duplicate.show(5)

Tabela po inner join:
+---+-------------+---+-------------+
|id1|random_value1|id2|random_value2|
+---+-------------+---+-------------+
|  1|          767|  1|          734|
|  2|          221|  2|          710|
|  3|          888|  3|          207|
|  4|          788|  4|          243|
|  5|          422|  5|          965|
+---+-------------+---+-------------+
only showing top 5 rows

Tabela po inner join po usunięciu duplikatów:
+----+-------------+-------------+
| id1|random_value1|random_value2|
+----+-------------+-------------+
|  26|          274|          812|
|  29|          384|          552|
| 474|          284|          774|
| 964|          170|          704|
|1677|          199|          480|
+----+-------------+-------------+
only showing top 5 rows

Tabela po left join:
+---+-------------+---+-------------+
|id1|random_value1|id2|random_value2|
+---+-------------+---+-------------+
|  1|          767|  1|          734|
|  2|          221|  2|          710|
|  3|         